# Imports

In [116]:
import numpy as np
import pandas as pd
# SimFin database
import simfin as sf
# Yahoo Finance API
from yahoo_fin import stock_info as si
import yfinance as yf
# full-screen notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Functions

In [9]:
# gives a list of all nadaq traded ticker symbols
def nasdaq_tickers():
    return si.tickers_nasdaq()

# gives a list of all ticker symbols in the S&P500 index
def snp500_tickers():
    return si.tickers_sp500()

# gives an income statement (is), or balance sheet (bs), or cashflow statement either annual, or quarterly, or TTM for a list of stock tickers
def statement(tickers, statement = 'is', variant = 'quarterly'):
    with open('API.txt') as f:
        lines = f.readlines()
    sf.set_api_key(str(lines[0]))
    sf.set_data_dir('~/simfin_data/')
    if statement == 'is':
        df = sf.load_income(variant = variant, market = 'us')
    elif statement == 'bs':
        df = sf.load_balance(variant = variant, market = 'us')
    else: df = sf.load_cashflow(variant = variant, market = 'us')
    df = df.reset_index('Ticker')
    return df[df['Ticker'].isin(tickers)].drop(['Currency', 'SimFinId', 'Publish Date',
                                                'Restated Date', 'Fiscal Year', 'Fiscal Period', 'Shares (Basic)', 'Shares (Diluted)'], axis = 1).replace({np.nan: None})

# Code section

In [3]:
tickers = snp500_tickers()

In [12]:
BS = statement(tickers, statement = 'bs', variant = 'annual')
IS = statement(tickers, variant = 'annual')

Dataset "us-balance-annual" on disk (6 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (0 days old).
- Loading from disk ... Done!


In [114]:
BS

,Ticker,"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",Long Term Investments & Receivables,Other Long Term Assets,Total Noncurrent Assets,Total Assets,...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
Report Date,,,,,,,,,,,,,,,,,,,,,
2017-10-31,A,2678000000.0,724000000.0,575000000.0,4169000000.0,757000000.0,138000000.0,3362000000.0,4257000000.0,8426000000,...,210000000.0,1263000000.0,1801000000.0,2328000000.0,3591000000,5303000000.0,None,-126000000.0,4835000000.0,8426000000
2018-10-31,A,2247000000.0,776000000.0,638000000.0,3848000000.0,822000000.0,68000000.0,3803000000.0,4693000000.0,8541000000,...,0.0,1171000000.0,1799000000.0,2799000000.0,3970000000,5311000000.0,None,-336000000.0,4571000000.0,8541000000
2019-10-31,A,1382000000.0,930000000.0,679000000.0,3189000000.0,850000000.0,102000000.0,5311000000.0,6263000000.0,9452000000,...,616000000.0,2080000000.0,1791000000.0,2624000000.0,4704000000,5280000000.0,None,-18000000.0,4748000000.0,9452000000
2020-10-31,A,1441000000.0,1038000000.0,720000000.0,3415000000.0,845000000.0,158000000.0,5209000000.0,6212000000.0,9627000000,...,75000000.0,1467000000.0,2284000000.0,3287000000.0,4754000000,5314000000.0,None,81000000.0,4873000000.0,9627000000
2021-10-31,A,1575000000.0,1172000000.0,830000000.0,3799000000.0,945000000.0,185000000.0,5776000000.0,6906000000.0,10705000000,...,None,1708000000.0,2729000000.0,3608000000.0,5316000000,5323000000.0,None,348000000.0,5389000000.0,10705000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31,ZTS,1564000000.0,998000000.0,1427000000.0,4217000000.0,1435000000.0,None,2934000000.0,4369000000.0,8586000000,...,None,1094000000.0,4953000000.0,5706000000.0,6800000000,1018000000.0,-852000000.0,2109000000.0,1786000000.0,8586000000
2018-12-31,ZTS,1701000000.0,1036000000.0,1391000000.0,4399000000.0,1658000000.0,None,4720000000.0,6378000000.0,10777000000,...,9000000.0,1223000000.0,6443000000.0,7369000000.0,8592000000,1031000000.0,-1487000000.0,3270000000.0,2185000000.0,10777000000
2019-12-31,ZTS,1934000000.0,1086000000.0,1410000000.0,4748000000.0,1940000000.0,None,4857000000.0,6797000000.0,11545000000,...,500000000.0,1806000000.0,5947000000.0,7031000000.0,8837000000,1049000000.0,-2042000000.0,4427000000.0,2708000000.0,11545000000


In [110]:
def data_formatting(S):
    data = [np.repeat([i for i in S['Ticker'].T], len(S.columns) - 1), [i for i in S.columns if i != 'Ticker']*S.shape[0], sum(np.array(S.drop('Ticker', axis = 1)).tolist(), [])]
    if np.array(data).shape[0] == 3:
        data = np.array(data).T.tolist()
    df = pd.DataFrame(data, columns = ['Ticker', 'Financial', 'Value'], index = np.repeat([i for i in S.index], len(S.columns) - 1))
    return df

In [113]:
data_formatting(BS).head(40)

,Ticker,Financial,Value
2017-10-31,A,"Cash, Cash Equivalents & Short Term Investments",2.678000e+09
2017-10-31,A,Accounts & Notes Receivable,7.240000e+08
2017-10-31,A,Inventories,5.750000e+08
2017-10-31,A,Total Current Assets,4.169000e+09
2017-10-31,A,"Property, Plant & Equipment, Net",7.570000e+08
2017-10-31,A,Long Term Investments & Receivables,1.380000e+08
2017-10-31,A,Other Long Term Assets,3.362000e+09
2017-10-31,A,Total Noncurrent Assets,4.257000e+09
2017-10-31,A,Total Assets,8.426000e+09
2017-10-31,A,Payables & Accruals,3.050000e+08


In [115]:
data_formatting(BS).to_excel('BS.xlsx')
data_formatting(IS).to_excel('IS.xlsx')

In [117]:
def stock_info(tickers):
    stocks = yf.Tickers(' '.join(tickers).lower())
    info = ['longName', 'industry', 'sector']
    info_short = ['Name', 'Industry', 'Sector']
    table = []
    for i in tickers:
        table.append([stocks.tickers[i].info.get(key, float('nan')) for key in info])
    return pd.DataFrame(table, index = tickers, columns = info_short)

In [123]:
companies = stock_info(tickers)

In [124]:
companies

,Name,Industry,Sector
A,"Agilent Technologies, Inc.",Diagnostics & Research,Healthcare
AAL,American Airlines Group Inc.,Airlines,Industrials
AAP,"Advance Auto Parts, Inc.",Specialty Retail,Consumer Cyclical
AAPL,Apple Inc.,Consumer Electronics,Technology
ABBV,AbbVie Inc.,Drug Manufacturers—General,Healthcare
...,...,...,...
YUM,"Yum! Brands, Inc.",Restaurants,Consumer Cyclical
ZBH,"Zimmer Biomet Holdings, Inc.",Medical Devices,Healthcare
ZBRA,Zebra Technologies Corporation,Communication Equipment,Technology
ZION,"Zions Bancorporation, National Association",Banks—Regional,Financial Services


In [125]:
companies.to_excel('S&P500.xlsx')